In [3]:
import zabbixapi

In [4]:
from openpyxl import *
from openpyxl.styles import *
from zabbix.api import ZabbixAPI
from datetime import datetime
from datetime import timedelta
import time
import os
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool 

In [5]:
zuser="api_user" # имя пользователя
zpass="111" # пароль пользователя
zurl="http://192.168.200.230"   # адрес сервера
zapi = ZabbixAPI(url=zurl, user=zuser, password=zpass)
groupIds=['10'] # id групп хостов

In [6]:
def get_data_from_productive_ifaces(gid, dfrom, dtill):
    rd={}
    hosts = zapi.host.get(output=['host','hostid'], groupids=gid)
    #print(hlist)
    #print(hosts)
    for h in hosts:
        
        host=h['hostid']
        hostname=h['host']
        rd[hostname]={}
        in_item = zapi.item.get(output='extend', hostids=host, search={'key_': 'net.if.in[bond1]'})
        out_item = zapi.item.get(output='extend', hostids=host, search={'key_': 'net.if.out[bond1]'})
        print(hostname,in_item[0]['key_'],in_item[0]['itemid'],out_item[0]['key_'],out_item[0]['itemid'])
        initem=in_item[0]['itemid']
        outitem=out_item[0]['itemid']
        print(initem,outitem)
        in_history = zapi.history.get(output='extend',itemids=initem, time_from=dfrom, time_till=dtill)
        out_history = zapi.history.get(output='extend',itemids=outitem, time_from=dfrom, time_till=dtill)
        print(hostname)
        inlist=[int(i['value']) for i in in_history]
        outlist=[int(i['value']) for i in out_history]
        inmean=sum(inlist)/len(inlist)
        outmean=sum(outlist)/len(outlist)
        print(inmean,outmean)
        print(np.std(inlist),np.std(outlist))
        rd[hostname]['std_in']=np.std(inlist)
        rd[hostname]['std_out']=np.std(outlist)
        rd[hostname]['mean_in']=inmean
        rd[hostname]['mean_out']=outmean
        
              #print(in_history)
        #print(out_history)
    return rd 

def add_sheet(d,groupname,ofile):
    wb = Workbook()
    'функция добавления данных в excel формат'
    if wb.active.title == "Sheet":
        wb.remove_sheet(wb.active)
    ws=wb.create_sheet(title=groupname)
    i=0
    ws.merge_cells('A1:E1')
    ws['A1'] = "Группа хостов:  "+groupname
    ws.cell(column=1, row=2+i, value='Хост')
    ws.cell(column=2, row=2+i, value='Входящий трафик, Мб/c')
    ws.cell(column=3, row=2+i, value='Исходящий трафик, Мб/c')
    for host in sorted(d.keys()):
        #print(host)
        ws.cell(column=1,row=3+i, value=host)
        ws.cell(column=2, row=3+i, value=(d[host]['mean_in']/1024/1024))
        ws.cell(column=3, row=3+i, value=(d[host]['mean_out']/1024/1024))
        ws.cell(column=4, row=3+i, value=d[host]['std_in'])
        ws.cell(column=5, row=3+i, value=d[host]['std_'])
        i = i + 1
    
    
    '''
    data = ws.get_cell_collection()
    dict2 = dict()
    for i in data:
        key = i.column
        subkey = i.row
        value = i.coordinate
        if key in dict2:
            dict2[key][subkey] = value
        else:
            dict2[key] = {}
            dict2[key][subkey] = value
    #print(dict2)
    for col in dict2:
        data1 = []
        for cell in dict2[col]:
            data1.append(dict2[col][cell])
        column_widths = []
        for cell in data1:
            #print(cell)
            cellvalue = ws.cell(cell).value
            ws.cell(cell).alignment=Alignment(horizontal='center')
            column_widths.append(len(cellvalue))
            #print(max(column_widths))
            ws.column_dimensions[col].width = max(column_widths) + 5
            '''
    wb.save(ofile)


In [7]:
dnow=datetime.now()
dnow_ts = time.mktime(dnow.timetuple())
dweekago = dnow - timedelta(days=7)
dweekago_ts = time.mktime(dweekago.timetuple())
dweekago_ts, dnow_ts

(1505755737.0, 1506360537.0)

In [8]:
#data_dict = get_data_from_productive_ifaces(gid=groupIds,dfrom=dweekago_ts,dtill=dnow_ts)

In [9]:
#data_dict

In [10]:
groupname=zapi.hostgroup.get(output=("groupid","name"),groupids=groupIds)[0]['name']

In [11]:
groupname

'4. Вычислительные узлы'

In [12]:
#add_sheet(data_dict,groupname, ofile='./test.xlsx')

In [13]:
#from multiprocessing.dummy import Pool as ThreadPool 

In [14]:
def get_data_from_productive_ifaces_multithreading(gid, dfrom, dtill):
    pool=ThreadPool(20)
    global f 
    f = dfrom
    global t 
    t = dtill
    hosts = zapi.host.get(output=['host','hostid'], groupids=gid)
    #print(hlist)
    #print(hosts)
    results=pool.map(get_data_hosts,hosts)
    return results
def get_data_hosts(h):
    host=h['hostid']
    hostname=h['host']
    print(hostname,'started')
    #rd[hostname]={}
    in_item = zapi.item.get(output='extend', hostids=host, history=0, search={'key_': 'system.cpu.util[,idle]'})
    #print(in_item)
    out_item = zapi.item.get(output='extend', hostids=host,history=0, search={'key_': 'vm.memory.size[available]'})
    #print(hostname,in_item[0]['key_'],in_item[0]['itemid'],out_item[0]['key_'],out_item[0]['itemid'])
    initem=in_item[0]['itemid']
    outitem=out_item[0]['itemid']
    #print(initem,outitem)
    in_history = zapi.history.get(output='extend',itemids=initem, time_from=f, time_till=t)
    out_history = zapi.history.get(output='extend',itemids=outitem, time_from=f, time_till=t)
    #print(hostname)
    inlist=[int(i['value']) for i in in_history]
    outlist=[int(i['value']) for i in out_history]
    inmean=sum(inlist)/len(inlist)
    inmean = 100 - inmean
    outmean=sum(outlist)/len(outlist)
    #print(inmean,outmean)
    #print(np.std(inlist),np.std(outlist))
    #rd[hostname]['std_in']=np.std(inlist)
    #rd[hostname]['std_out']=np.std(outlist)
    #rd[hostname]['mean_in']=inmean
    #rd[hostname]['mean_out']=outmean
              #print(in_history)
        #print(out_history)
    print(hostname,'finished')
    return [hostname,inmean,outmean]

In [ ]:
%%time
data_dict_1 = get_data_from_productive_ifaces_multithreading(gid=groupIds,dfrom=dweekago_ts,dtill=dnow_ts)

cn-e4ba.egron.tnxcn-d720.egron.tnxcn-d724.egron.tnxcn-da1a.egron.tnxcn-da4a.egron.tnxcn-da14.egron.tnxcn-da16.egron.tnxcn-da18.egron.tnxcn-da22.egron.tnxcn-da32.egron.tnxcn-da44.egron.tnxcn-da46.egron.tnxcn-da48.egron.tnxcn-da1e.egron.tnxcn-da74.egron.tnxcn-daa0.egron.tnxcn-daa6.egron.tnxcn-db0a.egron.tnxcn-db08.egron.tnxcn-db3e.egron.tnx                    startedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstartedstarted



















cn-db26.egron.tnx started
cn-db28.egron.tnx started
cn-db32.egron.tnx started
cn-db38.egron.tnx started
cn-db74.egron.tnxcn-db76.egron.tnx  startedstarted

cn-dc6c.egron.tnx started
cn-dc7c.egron.tnx started
cn-dc8c.egron.tnx started
cn-dc8e.egron.tnxcn-dc9a.egron.tnx  startedstarted

cn-dc84.egron.tnx started
cn-dca2.egron.tnx started
cn-dca4.egron.tnx started
cn-dca6.egron.tnx started
cn-dca8.egron.tnx started
cn-dcaa.egron.tnx started
cn-dce2.egron.tnx started
cn-dce8.egro

In [ ]:
data_dict_1